# CosyVoice Kaggle Notebook (Conda 环境版)

使用 Miniconda + 持久化目录部署 CosyVoice

**特性：**
- ✅ 使用 Conda 环境（Python 3.10）
- ✅ 持久化存储（环境 + 模型保存到 /kaggle/working）
- ✅ 避免重复下载和安装
- ✅ 支持会话中断后继续

**Repository:** https://github.com/infinite-gaming-studio/CosyVoice

## 0. 配置持久化路径

定义工作目录（所有数据都会保存在这里，会话结束后可保存为 Dataset）

In [ ]:
import os

# Kaggle 持久化工作目录
WORK_DIR = '/kaggle/working/cosyvoice_env'
CONDA_DIR = f'{WORK_DIR}/miniconda3'
REPO_DIR = f'{WORK_DIR}/CosyVoice'
MODEL_DIR = f'{WORK_DIR}/models'

# 创建目录
os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

print(f"工作目录: {WORK_DIR}")
print(f"Conda 目录: {CONDA_DIR}")
print(f"代码目录: {REPO_DIR}")
print(f"模型目录: {MODEL_DIR}")

## 1. 安装 Miniconda（仅需执行一次）

In [ ]:
import os

if not os.path.exists(f'{CONDA_DIR}/bin/conda'):
    print("正在安装 Miniconda...")
    !wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O /tmp/miniconda.sh
    !bash /tmp/miniconda.sh -b -p {CONDA_DIR}
    !{CONDA_DIR}/bin/conda init bash
    !{CONDA_DIR}/bin/conda clean -afy
    print("Miniconda 安装完成！")
else:
    print("Miniconda 已存在，跳过安装")

## 2. 克隆代码仓库

In [ ]:
if not os.path.exists(f'{REPO_DIR}/.git'):
    print("正在克隆仓库...")
    !cd {WORK_DIR} && git clone --recursive https://github.com/infinite-gaming-studio/CosyVoice.git
    print("仓库克隆完成！")
else:
    print("仓库已存在，更新中...")
    !cd {REPO_DIR} && git pull
    !cd {REPO_DIR} && git submodule update --init --recursive
    print("仓库更新完成！")

## 3. 创建 Conda 环境并安装依赖（仅需执行一次）

In [ ]:
# 检查环境是否已存在
env_exists = os.path.exists(f'{CONDA_DIR}/envs/cosyvoice')

if not env_exists:
    print("创建 Conda 环境...")
    !{CONDA_DIR}/bin/conda create -n cosyvoice python=3.10 -y
    
    # 安装系统依赖
    print("安装系统依赖...")
    !apt-get update -qq
    !apt-get install -y -qq sox libsox-dev build-essential
    
    # 安装 Python 依赖
    print("安装 Python 依赖（需要几分钟）...")
    !{CONDA_DIR}/envs/cosyvoice/bin/pip install -q -r {REPO_DIR}/requirements.txt --no-deps 2>&1 | tail -30
    
    # 额外安装关键依赖（确保兼容）
    !{CONDA_DIR}/envs/cosyvoice/bin/pip install -q torch==2.3.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu121
    !{CONDA_DIR}/envs/cosyvoice/bin/pip install -q transformers==4.51.3 modelscope gradio
    
    print("环境创建完成！")
else:
    print("Conda 环境已存在，跳过创建")

## 4. 下载模型（仅需执行一次）

In [ ]:
import sys
sys.path.insert(0, f'{REPO_DIR}')

# 使用 conda 环境的 Python
import subprocess

MODEL_NAME = 'Fun-CosyVoice3-0.5B-2512'  # 可选: 'CosyVoice2-0.5B', 'CosyVoice-300M'
MODEL_PATH = f'{MODEL_DIR}/{MODEL_NAME}'

if not os.path.exists(MODEL_PATH):
    print(f"正在下载 {MODEL_NAME} 模型...")
    
    # 使用 conda 环境的 Python 下载
    download_script = f'''
import sys
sys.path.insert(0, "{REPO_DIR}")
from modelscope import snapshot_download
snapshot_download("FunAudioLLM/{MODEL_NAME}", local_dir="{MODEL_PATH}")
print("下载完成！")
'''
    
    with open('/tmp/download_model.py', 'w') as f:
        f.write(download_script)
    
    result = subprocess.run(
        [f'{CONDA_DIR}/envs/cosyvoice/bin/python', '/tmp/download_model.py'],
        capture_output=True,
        text=True
    )
    print(result.stdout)
    if result.stderr:
        print("错误:", result.stderr)
else:
    print(f"模型 {MODEL_NAME} 已存在")

## 5. 验证安装

In [ ]:
# 验证 Python 版本和关键包
!{CONDA_DIR}/envs/cosyvoice/bin/python --version
!{CONDA_DIR}/envs/cosyvoice/bin/python -c "import torch; print(f'PyTorch: {torch.__version__}'); print(f'CUDA: {torch.cuda.is_available()}')"
!{CONDA_DIR}/envs/cosyvoice/bin/python -c "import transformers; print(f'Transformers: {transformers.__version__}')"

## 6. 测试推理

In [ ]:
# 创建测试脚本
test_script = f'''
import sys
sys.path.insert(0, "{REPO_DIR}")
sys.path.insert(0, "{REPO_DIR}/third_party/Matcha-TTS")

from cosyvoice.cli.cosyvoice import AutoModel
import torchaudio
import os

print("正在加载模型...")
model = AutoModel(model_dir="{MODEL_PATH}")
print(f"模型加载完成！采样率: {{model.sample_rate}}")

# 测试文本
text = "你好，欢迎使用 CosyVoice 语音合成系统。这是使用 Conda 环境在 Kaggle 上运行的测试。"

# 简单测试（无需 prompt 音频）
print("正在生成测试音频...")
for i, result in enumerate(model.inference_instruct2(text, "You are a helpful assistant.<|endofprompt|>", stream=False)):
    output_path = "{WORK_DIR}/test_output_{{i}}.wav"
    torchaudio.save(output_path, result['tts_speech'], model.sample_rate)
    print(f"已保存: {{output_path}}")

print("测试完成！")
'''

with open('/tmp/test_inference.py', 'w') as f:
    f.write(test_script)

# 运行测试
!{CONDA_DIR}/envs/cosyvoice/bin/python /tmp/test_inference.py

## 7. 启动 Web UI

运行后会显示公共访问链接（Gradio Live）

In [ ]:
# 启动 Web UI
!cd {REPO_DIR} && {CONDA_DIR}/envs/cosyvoice/bin/python webui.py \
    --port 50000 \
    --model_dir {MODEL_PATH} \
    --share

## 8. 持久化保存（可选）

如果你想在会话结束后保留环境和模型，可以打包保存：

In [ ]:
# 查看当前占用空间
!du -sh {WORK_DIR}/*

In [ ]:
# 导出 conda 环境配置（便于下次快速重建）
!{CONDA_DIR}/bin/conda env export -n cosyvoice > {WORK_DIR}/cosyvoice_environment.yml
print(f"环境配置已保存到: {WORK_DIR}/cosyvoice_environment.yml")

In [ ]:
# 如果需要，可以打包模型（注意：模型较大，约 2-4GB）
# !tar -czf /kaggle/working/cosyvoice_model.tar.gz -C {MODEL_DIR} {MODEL_NAME}
# print("模型已打包，可以从右侧 Output 下载")

---

## 📋 后续会话快速恢复

如果 Kaggle 会话中断，只需重新运行 **第0节** 和 **第7节**（Web UI），无需重复安装。